In [2]:
#imports
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
import tensorflow as tf

2024-01-10 13:49:03.257857: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
df = pd.read_csv('traindata.csv')
df.head()

,redCorner,blueCorner,winner,referee,venue,title_fight,billing,redCorner_wins,blueCorner_wins,redCorner_losses,...,blueCorner_subs_attempted,redCorner_sig_str_landed_per_minute,blueCorner_sig_str_landed_per_minute,fightTime,redCorner_sig_str_absorbed_per_minute,blueCorner_sig_str_absorbed_per_minute,redCorner_sig_str_defense,blueCorner_sig_str_defense,redCorner_takedown_defense,blueCorner_takedown_defense
0,0,2,0,161,139,1,0,21,17,3,...,0.0,2.280000,1.760000,25.000000,1.760000,2.280000,0.650794,0.472222,0.80,0.333333
1,0,2,0,162,139,1,1,26,15,5,...,0.0,3.800000,4.440000,25.000000,4.440000,3.800000,0.455882,0.391026,1.00,0.428571
2,0,2,0,87,139,0,2,17,17,6,...,0.0,2.818792,1.610738,9.933333,1.610738,2.818792,0.360000,0.222222,1.00,0.800000
3,0,2,2,31,139,0,2,25,20,9,...,0.0,4.133333,7.066667,15.000000,7.066667,4.133333,0.357576,0.409524,0.75,1.000000
4,0,2,0,161,139,0,2,18,16,4,...,0.0,1.025641,0.000000,1.950000,0.000000,1.025641,1.000000,0.800000,1.00,1.000000


In [4]:
#columns that need one-hot encoding
columns2encode = ['referee', 'venue', 'billing', 'redCorner_nation', 'blueCorner_nation']

#encode
encoder = OneHotEncoder(sparse=False)
data2encode = df[columns2encode]
encoded_cols = encoder.fit_transform(data2encode)

#create df for encoded columns
encoded_df = pd.DataFrame(encoded_cols, columns=encoder.get_feature_names_out(columns2encode))

#drop original columns
df.drop(columns2encode, axis=1, inplace=True)

#add enocded columns
df = pd.concat([df, encoded_df], axis=1)

df.head()

/Users/maxbraithwaite/Desktop/UFCModel/UFCModel/UFCModelGit/.venv/lib/python3.8/site-packages/sklearn/preprocessing/_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


,redCorner,blueCorner,winner,title_fight,redCorner_wins,blueCorner_wins,redCorner_losses,blueCorner_losses,redCorner_draws,blueCorner_draws,...,blueCorner_nation_101,blueCorner_nation_102,blueCorner_nation_103,blueCorner_nation_104,blueCorner_nation_105,blueCorner_nation_106,blueCorner_nation_107,blueCorner_nation_108,blueCorner_nation_109,blueCorner_nation_110
0,0,2,0,1,21,17,3,3,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0,2,0,1,26,15,5,6,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0,2,0,0,17,17,6,0,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0,2,2,0,25,20,9,3,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0,2,0,0,18,16,4,1,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
#x,y split
target_column = 'winner'
y = df[target_column]
x = df.drop(target_column, axis=1)

#create train test split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)


# Define the deep learning model
model = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(x_train.shape[1],)),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid') 
])

#compile
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

#train
model.fit(x_train, y_train, epochs=1000, batch_size=124, validation_data=(x_test, y_test))

#accuracy
loss, accuracy = model.evaluate(x_test, y_test)
print(f"Loss: {loss}, Accuracy: {accuracy}")

Epoch 1/1000
40/40 [==============================] - 2s 15ms/step - loss: nan - accuracy: 0.5358 - val_loss: nan - val_accuracy: 0.5662
Epoch 2/1000
40/40 [==============================] - 0s 11ms/step - loss: nan - accuracy: 0.5738 - val_loss: nan - val_accuracy: 0.5662
Epoch 3/1000
40/40 [==============================] - 0s 9ms/step - loss: nan - accuracy: 0.5738 - val_loss: nan - val_accuracy: 0.5662
Epoch 4/1000
40/40 [==============================] - 0s 8ms/step - loss: nan - accuracy: 0.5738 - val_loss: nan - val_accuracy: 0.5662
Epoch 5/1000
40/40 [==============================] - 0s 6ms/step - loss: nan - accuracy: 0.5738 - val_loss: nan - val_accuracy: 0.5662
Epoch 6/1000
40/40 [==============================] - 0s 6ms/step - loss: nan - accuracy: 0.5738 - val_loss: nan - val_accuracy: 0.5662
Epoch 7/1000
40/40 [==============================] - 0s 7ms/step - loss: nan - accuracy: 0.5738 - val_loss: nan - val_accuracy: 0.5662
Epoch 8/1000
40/40 [==========================